# Aula 09.Ex01 - Malhas e Texturas - Mapeamento de Texturas

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [ ]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
!pip install glm
!pip install pillow
import glm
import math
from PIL import Image

### Inicializando janela

In [ ]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [ ]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 mat_transform;        
        
        void main(){
            gl_Position = mat_transform * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [ ]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [ ]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [ ]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [ ]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [ ]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [ ]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [ ]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

* Arquivo do tipo [Wavefront](https://en.wikipedia.org/wiki/Wavefront_.obj_file): possuem informações para representação de objetos 3D como posição dos vértices de malha e textura. A seguir, a explicação, em ordem de uso dentro do arquivo, do que é cada item
    * `v`: vértices
    * `vt`: vértices de textura
    * `vn`: vértices normal (para iluminação e sombreamento)
    * `vp`: vértices paramétricos (para superfícies continuas e específicas - não usamos)
    * `f`: representa a função de como construir o polígono
        * `a/b/c`: `a` é a posição do vértice, `b` posição do vértice textura e `c` informação sobre iluminação

In [ ]:
# Função: carrega o arquivo Wavefront
# Entrada: nome do arquivo
# Saida: estrutura que armazena o elemento (vertices, textura e faces)
def load_model_from_file(filename):
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # Abre o arquivo obj (wavefront) para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        # Se for comentário, ignore esta linha e use a próxima
        if line.startswith('#'): continue

        # Quebra a linha por espaço
        values = line.split()
        # Se não há informações na linha, ignore esta linha e use a próxima
        if not values: continue

        # Recupera as informações
        ### Armazena coordenadas dos vertices do elemento no vetor vertices
        if values[0] == 'v':
            vertices.append(values[1:4])
        ### Armazena coordenadas das texturas no vetor texture_coords
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])
        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        ### Armazena informações sobre a construção das faces
        elif values[0] == 'f':
            face = []
            face_texture = []
            # Para cada elemento da linha que define a função
            for bloco in values[1:]:
                # Separa o elemento em vetor de elementos separando os números que são separados por /
                positions = bloco.split('/')
                # Adiciona o primeiro número na face (que representa o número da linha que encontra-se um vértice para da figura)
                face.append(int(positions[0]))
                # Se o vetor com elementos separados por / for maior ou igual que dois
                # Se o segundo número do elemento for maior do que zero
                if len(positions) >= 2 and len(positions[1]) > 0:
                    # Adicione o segundo número na face de textura (que representa o número da linha que encontra-se um vértice de textura da figura)
                    face_texture.append(int(positions[1]))
                else:
                    # Se não for maior ou igual a dois ou não for maior que zero, coloque zero na textura
                    face_texture.append(0)
            # Após conseguir, provavelmente, os três valores para face, os três valores para textura e o tipo de material, insira na faces
            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

### Não entendi essa parte do código

In [ ]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Vamos carregar cada modelo e definir funções para desenhá-los

In [ ]:
CAIXA_PATH : str = 'caixa/caixa.obj'
modelo = load_model_from_file(CAIXA_PATH)

### Coletando vetor de vertices e textura

def get_vertices_textures(model):
    vertices_list = []    
    textures_coord_list = []
    # Para cada um das faces (num_line(v), num_line(vt), material)
    for face in model['faces']:
        # Para cada um dos números que representa a linha do vértice
        for vertice_id in face[0]: # Pega o valor a coordenada do vértice
            vertices_list.append( model['vertices'][vertice_id-1] )
        # Para cada um dos números que representa a linha da coordenada da textura
        for texture_id in face[1]:  # Pega o valor a coordenada da textura
            textures_coord_list.append( model['texture'][texture_id-1] )
    return vertices_list, textures_coord_list

vertices_list, textures_coord_list = get_vertices_textures(modelo)

In [ ]:

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0,'caixa/caixa2.jpg')


In [ ]:
# modelo = load_model_from_file('terreno/terreno2.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
# for face in modelo['faces']:
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         textures_coord_list.append( modelo['texture'][texture_id-1] )
# print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(1,'terreno/pedra.jpg')


In [ ]:
# modelo = load_model_from_file('casa/casa.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
# for face in modelo['faces']:
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         textures_coord_list.append( modelo['texture'][texture_id-1] )
# print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(1,'casa/casa.jpg')


In [ ]:
# modelo = load_model_from_file('monstro/monstro.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
# for face in modelo['faces']:
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         textures_coord_list.append( modelo['texture'][texture_id-1] )
# print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(3,'monstro/monstro.jpg')


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [ ]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

###  Enviando coordenadas de vértices para a GPU

* Vale ressaltar que na função `glBufferdata`, estamos usando o argumento `GL_STATIC_DRAW` e não `GL_DYNAMIC_DRAW`.
    * `GL_STATIC_DRAW`: otimiza quando os dados do buffer não vão ficar mudando constantemente
    * `GL_DYNAMIC_DRAW`: otimiza quando os dados do buffer vão ficar mudando constantemente

In [ ]:
#Finaliza a modelagem dos dados de vértices
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [ ]:
#Finaliza a modelagem dos dados de texturas
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)])
textures['position'] = textures_coord_list

#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
#Subindo os dados de textura para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das texturas
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "texture_coord" e habilitando
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
#Linkando dados ao atributo "texture_coord"
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo

* PAREI AQUI


In [ ]:
def desenha_caixa():
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 36) ## renderizando

In [ ]:
# def desenha_terreno():
#     # aplica a matriz model
#     # rotacao
#     angle = 0.0;
#     r_x = 0.0; r_y = 0.0; r_z = 1.0;
#     # translacao
#     t_x = 0.0; t_y = -1.01; t_z = 0.0;
#     # escala
#     s_x = 20.0; s_y = 20.0; s_z = 20.0;
#     mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
#     loc_model = glGetUniformLocation(program, "model")
#     glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
#     #define id da textura do modelo
#     glBindTexture(GL_TEXTURE_2D, 1)
#     # desenha o modelo
#     glDrawArrays(GL_TRIANGLES, 36, 42-36) ## renderizando

In [ ]:
# def desenha_casa():
#     #define id da textura do modelo
#     glBindTexture(GL_TEXTURE_2D, 1)
#     # desenha o modelo
#     glDrawArrays(GL_TRIANGLES, 36, 1434) ## renderizando

In [ ]:
# def desenha_monstro(rotacao_inc):
#     #define id da textura do modelo
#     glBindTexture(GL_TEXTURE_2D, 3)
#     # desenha o modelo
#     glDrawArrays(GL_TRIANGLES, 1476, 7584-1476) ## renderizando

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [ ]:
polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 0.2
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
    
glfw.set_key_callback(window,key_event)


### Nesse momento, nós exibimos a janela!


In [ ]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
# Habilita 3D
glEnable(GL_DEPTH_TEST)

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

rotacao_inc = 0
d = 0 
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    d -= 0.001 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_y)
    mat_transform = multiplica_matriz(mat_rotation_x,mat_transform)
    mat_scale       = np.array([    0.1, 0.0, 0.0, 0.0, 
                                    0.0, 0.1, 0.0, 0.0, 
                                    0.0, 0.0, 0.03, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)    
    mat_transform = multiplica_matriz(mat_scale,mat_transform)

    loc_mat_transform = glGetUniformLocation(program, "mat_transform")
    glUniformMatrix4fv(loc_mat_transform, 1, GL_FALSE, mat_transform) 

    desenha_caixa()   
#     desenha_terreno()
#     desenha_casa()
    
#     rotacao_inc += 0.1
#     desenha_monstro(rotacao_inc)
  
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

* Adicione mais 2 modelos no cenário com suas respectivas texturas. Procure em repositórios abertos/gratuitos por modelos no formato Wavefront (extensão .obj). Verifique se o conteúdo das faces do modelo é baseado em triângulos. Verifique se o modelo acompanha alguma imagem (.jpg, png, etc) com a textura. Evite modelos compostos por múltiplos objetos/texturas.

* Coloque um cubo para "encapsular" todo o seu cenário. A face inferior do cubo será seu terreno. A face superior será o céu. As faces laterais serão horizontes. Crie um único arquivo de textura (imagem png ou jpg) com todas as faces. No arquivo .obj do modelo, define as coordenadas de textura para cada triângulo.